# CLEAN Algorithm

Based on "The CLEAN type algorithms for radar signal processing" by [Kulpa (2008)](https://ieeexplore.ieee.org/document/4669567).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pluto_sdr_pr.ioutils
import pluto_sdr_pr.signal
import pluto_sdr_pr.processing

In [ ]:
c = 3e8
cpi = 0.1
num_frames = 12
time_skip_after_sync = 10

In [ ]:
ref_file_path = "data/pluto_a_ref.2021-08-13T17_56_09_570.sdriq"
surv_file_path = "data/pluto_b_surv.2021-08-13T17_56_12_704.sdriq"

ref = pluto_sdr_pr.ioutils.SdriqSampleIO(ref_file_path)
surv = pluto_sdr_pr.ioutils.SdriqSampleIO(surv_file_path)
enb = pluto_sdr_pr.signal.ENodeB(6)
mss = pluto_sdr_pr.signal.MultiSignalStream()

cell_id, pss_correlations, _ = mss.start_synchronized(
    ref,
    surv,
    enb=enb,
    num_frames=num_frames,
    pss_only=True,
)

_ = ref.seek(time_skip_after_sync * ref.sample_rate)
_ = surv.seek(time_skip_after_sync * surv.sample_rate)

In [ ]:
def calc_plot_parameters(sampling_rate, cpi, center_freq, max_distance=50e3, max_speed=280):
    num_samples = int(cpi * sampling_rate)
    max_delay = int(max_distance / c * sampling_rate)
    max_doppler = int(max_speed * cpi * center_freq / c)
    return (num_samples, max_delay, max_distance, max_doppler, max_speed)

In [ ]:
def plot_ambiguity(amb, sample_rate, center_frequency, ax = None):
    if ax == None:
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)

    ax.set_xlabel("bistatic range [km]")
    ax.set_ylabel("bistatic velocity [m/s]")
    xticks = np.linspace(0, 50e3, 6, endpoint=True)
    ax.set_xticks(xticks / c * sample_rate)
    ax.set_xticklabels(map(lambda x: f"{x // 1e3:.0f} km", xticks))

    yticks = np.linspace(-max_speed, max_speed, 9, endpoint=True)
    ax.set_yticks((yticks + max_speed) * cpi * center_frequency / c)
    ax.set_yticklabels(map(lambda y: f"{y:.0f} m/s", yticks))

    ax.imshow(np.log10(np.abs(amb.T)))

    return ax

In [ ]:
num_samples, max_delay, max_distance, max_doppler, max_speed = (
    calc_plot_parameters(ref.sample_rate, cpi, ref.center_frequency)
)

ref_samples, surv_samples = mss.read(num_samples=num_samples)

assert ref_samples.source == ref and surv_samples.source == surv

def ambfun(ref_samples, surv_samples):
    amb = pluto_sdr_pr.processing.fast_ambiguity(
        max_delay,
        max_doppler * 2 + 1,
        ref_samples,
        surv_samples,
    )

    f = plt.figure(figsize=(20, 10))
    ax = plot_ambiguity(
        amb,
        ref.sample_rate,
        ref.center_frequency,
        ax=f.add_subplot(1, 1, 1)
    )
    ax.set_title("Pre-CLEAN Range/Doppler Map")

    return amb

cleaned_surv_samples, _ = pluto_sdr_pr.processing.clean(
    ref_samples,
    surv_samples,
    ambfun
)

amb = pluto_sdr_pr.processing.fast_ambiguity(
    max_delay,
    max_doppler * 2 + 1,
    ref_samples,
    cleaned_surv_samples,
)

f = plt.figure(figsize=(20, 10))
ax = plot_ambiguity(
    amb,
    ref.sample_rate,
    ref.center_frequency,
    ax=f.add_subplot(1, 1, 1)
)
_ = ax.set_title("Post-CLEAN Range/Doppler Map")